# Introdução

O campo da física de altas energias busca compreender as quatro principais forças que regem as interações entre partículas. As informações necessárias para este estudo são coletadas a partir do estudo de uma classificação específica de partículas, que são produzidas em condições extremas.

A única forma conhecida de se gerar e analisar tais partículas é por meio de colisões realizadas em aceleradores de partículas, porém nem toda colisão gera dados relevantes. Na realidade, o Grande Colisor de Hadrons (LHC) gera em média três partículas de interesse (denominadas partículas exóticas) a cada $10^9$ colisões. As colisões restantes geram partículas denominadas background, e não são utilizadas.

Dada a quantidade de colisões realizada por hora, e a complexidade de se analisar os dados obtidos, há um grande interesse em desenvolver uma forma automatizada de se diferenciar partículas exóticas das background. Para realizar esta diferenciação, são utilizados 18 dados. Destes, 8 são dados cinéticos coletados por detectores dentro do próprio acelerador de partículas, enquanto os 10 restantes são calculados em função dos anteriores.

O dataset analizado, denominado SUSY, apresenta um total de cinco milhões de instâncias de partículas teóricas. Cada uma possui 18 variáveis numéricas representando os dados utilizados para a diferenciação entre partículas exóticas e background, e uma variável booleana que representa a **qual destas classificações** a partícula em questão se encaixa. Os dados do dataset SUSY foram gerados a partir de simulações Monte Carlo, ou seja, não são produto de colisões reais.

# Objetivos
## Geral
Leonardo
    Determinar modelo de predição para classificar se a particula é exotica ou não.
## Especificos
Leonardo
    Determinar a melhor metrica para avaliar os resultados exemplos (precisão, acurácia, falso positivo, falso negativo) e porque

# Informações do Dataset
info, head, tail, describe
Akiyoshi

# Algoritmos de Classificação

A equipe utilizou como guia inicial para a escolha do algorítmo de classificação a documentação fornecida pelo sklearn, pois compõe uma relação compacta das variáveis a serem consideradas a cerca dos dados e tipo de classificação do problema quanto ao mesmo ser de aprendizado não supervisionado ([documentacao](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html). A imagem guia está disponível abaixo.

![imgsklearn](https://preview.ibb.co/g4d0hS/sklearn.png)

A premissa primeira engloba o fato de o problema ser de classificação, tal que os dados possuem labels bem definidas, binárias, que classificam uma partícula em exótica ou background. Tendo em vista o conjunto de 5.000.000 de dados na base de dados SUSY, foi preciso realizar a leitura de apenas uma parte dos mesmos, com o intuito principalmente de que os membros da equipe se tornassem familiarizados com o processo. Por esse motivo, foi utilizado inicialmente o KNN ou KNeighborsClassifier. 

A princípio mostrou resultados razoáveis, como é possível ver [aqui](#knnAcuracia), entretanto, o número de dados processados não englobava todos os registros do dataset e notou-se que esse método não era recomendado para conjunto de dados maiores que 100k. A incompatibilidade relatada, pode ser observada devido ao fato de que com acréscimos no número de registros a serem processados o algorítmo se tornava demasiadamente lento, como é possível observar na seção [Tempo de Execução](#tempoExecucao). É importante salientar que a análise desse algorítmo foi realizada pela equipe considerando apenas as últimas 8 colunas, pois a equipe raciocinou que, como elas eram funções das 10 primeiras poderia apresentar uma maior independência entre si, os resultados apresentados aqui, consideram essa condição. A partir de então, a equipe optou por testar [SGDClassifier], primeira opção para um conjunto de dados dessa dimensão, notando-se que o algorítmo não apresentou o resultado esperado o grupo optou por utilizar o Kernel Aproximate e o GBC.

## Nearest Neighbors Classification

A classificação baseada em vizinhos é um tipo de aprendizado que se baseia em intâncias : não tenta construir um modelo interno geral, mas simplesmente armazena instâncias dos dados de treinamento. A classificação é calculada a partir de uma votação simples dos vizinhos mais próximos de cada ponto.

A classificação básica de vizinhos mais próximos usa pesos uniformes: o valor atribuído a um ponto é calculado a partir de uma votação simples por maioria  dos vizinhos mais próximos. Em algumas circunstâncias, é melhor ponderar os vizinhos de tal forma que os vizinhos mais próximos contribuam mais para o ajuste.

[Documentação](http://scikit-learn.org/stable/modules/neighbors.html#classification)


## KNeighborsClassifier

É um algorítmo classificador que implementa o voto entre os k vizinhos mais próximos. 

[Documentação](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)


KNN,SGD,KA,GBC colocar imagem do sklearnin de seleção dos algoritmos  
Rafael, Adrianne

# Análise das Features
Gesiel falar sobre as variações dos modelos e pré concluir
## Pré Processamento
    O pré processamento de dados é uma fase no aprendizado de maquina que busca avaliar a qualidade dos dados e suas relações, e se iniciar logo depois de os dados terem sido coletados e organizados em um conjunto de dados. Essa fase se dá pelo fato de existirem diversos problemas nos dados que precisam ser solucionados como por exemplo: dados faltando, dados corrompidos, dados irrelevantes e valores desconhecidos.
    
    Uma caracteristica que os algoritmos de AM, vem assumindo é determinar que os dados já vem pré processados, isso porque a comunidade de AM usam os repositorios de dados para obter conjuntos de dados para utilização e por essa razão não apresentam varios problemas que podem ser encontrados em dados coletados no mundo real e acabam não realizando a fase de pré processamento.
    
    Diante desde dilema e pelo fato de o dataset SUSY possuir um conjunto de dados de cinco milhoes de registro e 18 variaveis independentes, esta sessão possue por objetivo buscar reduzir as variaveis independentes(features) do dataset, afim de melhorar o processamento e os resultados obtidos após o processamento e analise dos resultados, dessa forma varias tecnicas de redução de featues juntamente com analise gráfica seram utiladas em conjunto para determinar se é possivel reduzir as features.
    
    Este objetivo foi proposto depois de uma analise da estrutura dos dados e visualização dos mesmos utilizando tecnicas de visualização de dados, foi observado que os dados já foram pré processados como pode ser observado: não existem dados faltando, não existem dados corrompidos e não existem valores corrompidos.
    
    Analisando dois trabalhos feitos utilizando este mesmo conjunto de dados(SUSY) disponíveis em: [Searching for Exotic Particles in ...](https://arxiv.org/pdf/1402.4735.pdf) e [Modelo preditivo de Susy ...](http://machinelearningandspark.blogspot.com.br/2016/07/susy-dataset.html), notou-se que os melhores resultados foram obtidos utilizando todas as features
    
    
## Redução de Features
Afonso, João Pedro
## Teste Redução
    Testar as features com os algoritmos SGD, GBC de classificação
    Afonso
## Conclusão Análise Features
    Afonso
Concluir que é melhor não reduzir as features

# conclusão
## Algoritmo utilizado
Thiago
## Resultados Obtidos
Thiago
    Apresentar resultados obtidos, gráficos e resultados da mensuração

## Referências

BATISTA, Gustavo Enrique de Almeida Prado Alves. Pré-processamento de dados em aprendizado de máquina supervisionado. 2003. Tese (Doutorado em Ciências de Computação e Matemática Computacional) - Instituto de Ciências Matemáticas e de Computação, Universidade de São Paulo, São Carlos, 2003. doi:10.11606/T.55.2003.tde-06102003-160219. Acesso em: 2018-04-16.